In [26]:
#initialise

import extractor
extractor.main()
fps = extractor.fps
resolution = extractor.resolution





FPS: 23.976024070180753
Resolution: {width}x{height}
Audio extracted to output_data/audio.mp3
Extracted 16959 frames.
Feature data has been collected!
Writing data to csv
Features saved to output_data/output_features.csv
Extraction completed!


In [27]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor  

unbiased = False

In [28]:
import pydub

df = pd.read_csv('output_data/output_features.csv')
df['frame_index'] = df.index
df['normalized_index']=df['frame_index']/len(df)

audio_full = pydub.AudioSegment.from_mp3('output_data/audio.mp3')

slice_paths = []

for i in range(len(df['timestamp'])):
    frame_start = df['timestamp'][i]
    if i >= len(df['timestamp'])-1:
        frame_end = len(audio_full)
    else:
        frame_end = df['timestamp'][(i+1)]
    slice:pydub.AudioSegment = audio_full[frame_start:frame_end]
    slice_path = f'output_slices/audio_slice_{frame_start*0.001}_{i}.wav'
    slice.export(slice_path, format='wav')
    slice_paths.append(slice_path)

    




df['audio_slice_path'] = slice_paths
if unbiased:
    df = df.sample(frac=1.0,random_state=42).reset_index(drop=True)
print(df[['frame_filename','frame_index','normalized_index','audio_slice_path']].head())

                       frame_filename  frame_index  normalized_index  \
0    output_frames/frame_0_time_0.jpg            0          0.000000   
1   output_frames/frame_1_time_41.jpg            1          0.000059   
2   output_frames/frame_2_time_83.jpg            2          0.000118   
3  output_frames/frame_3_time_125.jpg            3          0.000177   
4  output_frames/frame_4_time_166.jpg            4          0.000236   

                        audio_slice_path  
0    output_slices/audio_slice_0.0_0.wav  
1  output_slices/audio_slice_0.041_1.wav  
2  output_slices/audio_slice_0.083_2.wav  
3  output_slices/audio_slice_0.125_3.wav  
4  output_slices/audio_slice_0.166_4.wav  


In [29]:
# Check shape
print("DataFrame shape:", df.shape)

# Check first few rows
print(df.head())

# Check for missing rows or NaNs
print(df.isnull().sum())

# Check if you accidentally only have one row
if df.shape[0] <= 1:
    print("Warning: Only 1 row detected!")


X = df.drop(['timestamp','frame_filename','frame_index','normalized_index','audio_slice_path'], axis=1)
y = df['normalized_index']

DataFrame shape: (16959, 1517)
   timestamp                      frame_filename  hist_bin_0  hist_bin_1  \
0          0    output_frames/frame_0_time_0.jpg         1.0         0.0   
1         41   output_frames/frame_1_time_41.jpg         1.0         0.0   
2         83   output_frames/frame_2_time_83.jpg         1.0         0.0   
3        125  output_frames/frame_3_time_125.jpg         1.0         0.0   
4        166  output_frames/frame_4_time_166.jpg         1.0         0.0   

   hist_bin_2  hist_bin_3  hist_bin_4  hist_bin_5  hist_bin_6  hist_bin_7  \
0         0.0         0.0         0.0         0.0         0.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         0.0   
3         0.0         0.0         0.0         0.0         0.0         0.0   
4         0.0         0.0         0.0         0.0         0.0         0.0   

   ...  edge_993  edge_994  edge_995  edge_996  e

In [30]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

model.fit(X_train, y_train)


RandomForestRegressor(n_jobs=-1, random_state=42)

In [31]:
y_pred = model.predict(X_test)

In [32]:
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.4f}")
print(f"R2 Score: {r2:.4f}")

MSE: 0.0011
R2 Score: 0.9867


In [33]:
df['predicted_normalized_index'] = model.predict(X)

# sort by predicted order
df_sorted = df.sort_values(by='predicted_normalized_index')

print(df_sorted[['frame_filename','predicted_normalized_index']].head(10))


                          frame_filename  predicted_normalized_index
34  output_frames/frame_34_time_1418.jpg                    0.002038
35  output_frames/frame_35_time_1459.jpg                    0.002086
38  output_frames/frame_38_time_1584.jpg                    0.002230
36  output_frames/frame_36_time_1501.jpg                    0.002261
37  output_frames/frame_37_time_1543.jpg                    0.002492
43  output_frames/frame_43_time_1793.jpg                    0.002526
44  output_frames/frame_44_time_1835.jpg                    0.002581
45  output_frames/frame_45_time_1876.jpg                    0.002648
49  output_frames/frame_49_time_2043.jpg                    0.002892
50  output_frames/frame_50_time_2085.jpg                    0.002956


In [34]:
import cv2

output_video = cv2.VideoWriter('output_data/reconstructed.avi',
                               cv2.VideoWriter_fourcc(*'XVID'),
                               fps,
                               resolution)


for _, row in df_sorted.iterrows():
    frame = cv2.imread(row['frame_filename'])
    output_video.write(frame)

output_video.release()
print("Reconstructed Video Saved!!")

Reconstructed Video Saved!!


In [35]:
final_audio:pydub.AudioSegment = pydub.AudioSegment.empty()
for s in df['audio_slice_path']:
    sorted_slice = pydub.AudioSegment.from_wav(s)
    final_audio = final_audio.append(sorted_slice, crossfade=0)
final_audio.export('output_data/final_audio.mp3')

<_io.BufferedRandom name='output_data/final_audio.mp3'>

In [36]:
import subprocess

def combine_audio_video(reconstructed_video, audio_file, output_video):
    command = ['ffmpeg', '-i', reconstructed_video, '-i', audio_file, '-c:v', 'copy', '-c:a', 'aac', output_video, '-y']
    subprocess.run(command, check=True)
    print(f"Final video saved to {output_video}")

combine_audio_video('output_data/reconstructed.avi', 'output_data/final_audio.mp3','output_data/final_output.mp4')

Final video saved to output_data/final_output.mp4


In [37]:
print(df_sorted['audio_slice_path'])

34                  output_slices/audio_slice_1.418_34.wav
35                  output_slices/audio_slice_1.459_35.wav
38                  output_slices/audio_slice_1.584_38.wav
36       output_slices/audio_slice_1.5010000000000001_3...
37                  output_slices/audio_slice_1.543_37.wav
                               ...                        
16915          output_slices/audio_slice_705.496_16915.wav
16914    output_slices/audio_slice_705.4540000000001_16...
16913          output_slices/audio_slice_705.413_16913.wav
16912          output_slices/audio_slice_705.371_16912.wav
16916          output_slices/audio_slice_705.538_16916.wav
Name: audio_slice_path, Length: 16959, dtype: object
